In [1]:
import os

import dotenv

from time import time, sleep
from algosdk.future import transaction
from algosdk.logic import get_application_address

from swap.operations import create_swap_app, place_swap, cancel_swap, accept_swap, close_swap
from utils import *
from account import Account

In [2]:
dotenv.load_dotenv('.env')

client = get_algod_client(os.environ.get('ALGOD_URL'), os.environ.get('ALGOD_TOKEN'))

In [3]:
creator = Account.from_mnemonic(os.environ.get('CREATOR_MN'))
bidder = Account.from_mnemonic(os.environ.get('BUYER_MN'))
seller = Account.from_mnemonic(os.environ.get('SELLER_MN'))

print(f"Creator address: {creator.get_address()}")
print(f"Bidder address: {bidder.get_address()}")
print(f"Seller Address: {seller.get_address()}")

Creator address: 4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64
Bidder address: X7PMCPPRD2T35ZZRTPETYK53G5YWBKUNONVDY52WODP3N3YM7NGEF5HGWU
Seller Address: CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA


Alice is generating an example token...

In [4]:
asset1_id = int(os.environ.get("TOKEN_ID"))
asset2_id = int(os.environ.get("TOKEN2_ID"))
print(f"The token id is: {asset1_id}")
print(f"The token2 id is: {asset2_id}")

The token id is: 2989
The token2 id is: 3559


"Alice is creating auction smart contract that lasts 30 seconds to auction off token..."

In [5]:
deleteApps(client, [], creator)

In [6]:

appID = create_swap_app(
    client=client,
    creator=creator
)
print(f"App ID: {appID}")
print(f"App Address: {get_application_address(appID)}")

Waiting for confirmation...
Transaction P3LV2MM7TIBG6RF73IJPKPQ3HLYPEZJMBTPA76XFNYDNRGWL575Q confirmed in round 145439.
Waiting for confirmation...
Transaction 7O5XN77263P3R5JSVEGICYA34QXLIICTM3G2RELL4ZNZXONGKSHQ confirmed in round 145441.
App ID: 3919
App Address: GXTGO46CMJGIZ4FVU22LQPOE6A5QEQDQQETKWYLHAID7PQW6RGV5S6TADA


Alice is setting up and funding token auction...

In [7]:
sellerAsset1BalanceBefore = get_balances(client, seller.get_address())[asset1_id]
print("Alice's asset1 balance: ", sellerAsset1BalanceBefore)

sellerAsset2BalanceBefore = get_balances(client, seller.get_address())[asset2_id]
print("Alice's asset2 balance: ", sellerAsset2BalanceBefore)

bidderAsset1BalanceBefore = get_balances(client, bidder.get_address())[asset1_id]
print("Carla's asset1 balance: ", bidderAsset1BalanceBefore)

bidderAsset2BalanceBefore = get_balances(client, bidder.get_address())[asset2_id]
print("Carla's asset2 balance: ", bidderAsset2BalanceBefore)

Alice's asset1 balance:  49999999868200
Alice's asset2 balance:  60000
Carla's asset1 balance:  50000000030000
Carla's asset2 balance:  49999999940000


In [8]:
actualAppBalancesBefore = get_balances(client, get_application_address(appID))
print("The smart contract now holds the following:", actualAppBalancesBefore)

The smart contract now holds the following: {0: 100000}


In [9]:
offeringAmount = 10_000
acceptingAmount = 20_000


In [10]:

swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index="")
print("Carla wants to swap on asset1 ", offeringAmount, " to asset2 ", acceptingAmount)
print("Carla bid", swap_index)

Waiting for confirmation...
Transaction HGM5OLVK5JTCKO5ZBD5JJBQ7TVDYDG4DATBQBQO5XDFDLXJZXZTA confirmed in round 145443.
new address: CKS5KSBRTNINH6C7P35NXKSMGY3FHTVFE2TDSXUICW6RMJ76CQAISM7L7Y
new private_key: 6wRbeW750I8+51jDDRQ9nmaTgmfuGzo2JTyIDAyoU5USpdVIMZtQ0/hffvrbqkw2NlPOpSamOV6IFb0WJ/4UAA==
new passphrase: output hire device notice peanut physical delay flight reveal expand burden hazard chat thing industry brief shrug network bulk silk blossom tube pride above jacket
Waiting for confirmation...
Transaction OI7WHWZCKB3VAQXYAWQPHULA2ETVKWSP6KQGY4XKCXWQWSWHGQAA confirmed in round 145445.
Waiting for confirmation...
Transaction 3MG6X7UBB2VXGOALZSY5WIFPWPYCXUHXHJSCXZD6UCSO2L76GMJA confirmed in round 145447.
Waiting for confirmation...
Transaction UFZAIQPWTVFNNXSM6SVGMMQSY6RSNUXXNB5XBIJNH6KOUF6VUMZQ confirmed in round 145449.
token_txn: {'sender': 'CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA', 'fee': 1000, 'first_valid_round': 145441, 'last_valid_round': 146441, 'note':

In [11]:
# account_info = client.account_info(bidder.get_address())  
# print(account_info.get('apps-local-state', []))
# print(is_opted_in_app(client, appID, bidder.get_address()))

cancel_swap(client, appID, seller, swap_index)
print("Carla cancel swap", swap_index)

Waiting for confirmation...
Transaction IGEYCFOAIYSEGQPBPDYR5OD756QZUZLI2SSE4U4ZKNWMAQ7WDK2A confirmed in round 145453.
Carla cancel swap CKS5KSBRTNINH6C7P35NXKSMGY3FHTVFE2TDSXUICW6RMJ76CQAISM7L7Y


In [12]:
swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index="")
# replace swap
swap_index = place_swap(client=client, app_id=appID, offer=seller, offering_token_id=asset1_id, offering_token_amount=offeringAmount, accepting_token_id=asset2_id, accepting_token_amount=acceptingAmount, swap_index=swap_index)

new address: FKSZ6WXD3BIEPRRV4YK4QNZGPAF67WMSXPSA5APA6PTBTLKGPMKA5HZZLU
new private_key: wxhgJMRFxEGfbzAjlRSBF7Bmzj9m9k0l6xBzK/gEOXMqpZ9a49hQR8Y15hXINyZ4C+/ZkrvkDoHg8+YZrUZ7FA==
new passphrase: blur lesson banana fox mass spare language corn piece false scene accident sniff vicious cradle wait predict prosper mask rescue utility donkey oil about humble
Waiting for confirmation...
Transaction 5BC27GSKD3WK65P4NRURC2V5FR45O4TNEXRZXTRAEZVKU35QFUBQ confirmed in round 145455.
Waiting for confirmation...
Transaction R64QB7EYGUXPVEUNTCF7ANBQVYZJLFA372FR6WARGCBZPA7MXOBA confirmed in round 145457.
Waiting for confirmation...
Transaction RUDESPUW5HLETBBLMX7S2U44A47EPRFWPPCC3BLNUMZNIE7H3LMA confirmed in round 145459.
token_txn: {'sender': 'CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA', 'fee': 1000, 'first_valid_round': 145453, 'last_valid_round': 146453, 'note': None, 'genesis_id': 'sandnet-v1', 'genesis_hash': 'S46yr60YjyJzK3Fk0J8V7mzMIOSxovaXAVqCHlwPl1A=', 'group': None, 'lease': N

In [13]:
accept_swap(client, appID, bidder, swap_index)

offer address CN4EQ6QXFCDLR2ATDTRJMMFGLXSUS5TMLGLO7FJDBQRQGFUXTUA4LBVEIA
offering_token_id 2989
offering_token_amount 10000
accepting_token_id 3559
accepting_token_amount 20000
Waiting for confirmation...
Transaction QYBMOJ5G4XXVD7OJ4Q5S22IVU5PVVD4GGEJUOMEANNWELLBRXI5Q confirmed in round 145465.


True

Alice is closing out the auction....

In [14]:
sleep(5)

close_swap(client, appID, creator, [asset1_id, asset2_id])

b'assets' [2989, 3559]
b'accounts' ['4TT75274EBUAF46CITUL6HQQ4C4D3GO7GEOVRZSQZ35VXSGRVHJ376GD64', 'UCAX3M5HS7PWWLB6HDCM2O7JE4HC3RGBPNWFRP4SYU62ZS5NSS2GFOKIOU']
Waiting for confirmation...
Transaction UOCGJDS6ST3IGLNIAFURONFCUWJR4XZY7BLS7QZ7AZNQWPUY2K3A confirmed in round 145468.


In [15]:
actualAppBalances = get_balances(client, get_application_address(appID))
expectedAppBalances = {0: 0}
print("The smart contract now holds the following:", actualAppBalances)

#assert actualAppBalances == expectedAppBalances

The smart contract now holds the following: {0: 300000, 2989: 0, 3559: 0}


In [16]:
sellerAsset1BalanceAfter = get_balances(client, seller.get_address())[asset1_id]
print("Alice's asset1 balance: ", sellerAsset1BalanceAfter)

sellerAsset2BalanceAfter = get_balances(client, seller.get_address())[asset2_id]
print("Alice's asset2 balance: ", sellerAsset2BalanceAfter)

bidderAsset1BalanceAfter = get_balances(client, bidder.get_address())[asset1_id]
print("Carla's asset1 balance: ", bidderAsset1BalanceAfter)

bidderAsset2BalanceAfter = get_balances(client, bidder.get_address())[asset2_id]
print("Carla's asset2 balance: ", bidderAsset2BalanceAfter)

Alice's asset1 balance:  49999999858200
Alice's asset2 balance:  80000
Carla's asset1 balance:  50000000040000
Carla's asset2 balance:  49999999920000
